In [84]:
# Important statistical libraries
import numpy as np
from numpy import mean
from numpy import std
import pandas as pd
from scipy import stats
from scipy.stats import norm, skew, probplot
# Some graphing libraries
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from matplotlib import pyplot

In [72]:
# Import train dataset into dataframe
train = pd.read_csv('/Users/robertx/Desktop/_/Sophmore-Fall/FA21_Exponent_Project/Data/train.csv')
test = pd.read_csv('/Users/robertx/Desktop/_/Sophmore-Fall/FA21_Exponent_Project/Data/test.csv')

In [73]:
#Combining Variables
#Combine Bsmt Full Bathrooms with Bsmt Half Bathrooms
train['TotalBsmtBath'] = train['BsmtFullBath']+0.5*train['BsmtHalfBath']
#Combine Above Grade Full Bathrooms with Above Garde Half Bathrooms
train['TotalAGBath'] = train['FullBath'] + 0.5*train['HalfBath']
train['TotalBath'] = train['TotalAGBath'] + train['TotalBsmtBath']
#Calculate Hosue Age (YearSold - YearRemodAdd)
train['HouseAge'] = train['YrSold'] - train['YearRemodAdd']
#Total in house area sqft (garage? LowQualFinSF?)
train['TotalInHouseArea'] = train['1stFlrSF'] + train['2ndFlrSF'] + train['TotalBsmtSF']
#TotalPorchArea (wooddeck?)
train['TotalPorchArea'] = train['OpenPorchSF'] + train['EnclosedPorch'] + train['3SsnPorch'] + train['ScreenPorch']

non_num_vars = [
    'MSZoning', 'LandContour', 'Utilities',
    'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 
    'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 
    'GarageType', 'MiscFeature', 
    'SaleType', 'SaleCondition', 'Alley', 'Street', 'Landslope', 'Mosold', 'Yrsold'
]
rank_vars = [
    'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual',
    'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
    'Functional', 'GarageFinish', 'PavedDrive', 
    'CentralAir', 'LotShape', 'MSSubClass', 'OverallQual'
]
continuous_vars = [
    'LotFrontage', 'LotArea', 
    'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', 
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'PoolArea', 'MiscVal','TotalInHouseArea'
]
discrete_vars = [
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 
    'Fireplaces', 'GarageYrBlt',
    'GarageCars','TotalBath','HouseAge'
]
num_vars = discrete_vars+continuous_vars
num_vars_df = train.loc[:,[i for i in num_vars]]
num_vars_df

,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,TotalBath,HouseAge,LotFrontage,LotArea,...,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,PoolArea,MiscVal,TotalInHouseArea
0,3,1,8,0,2003.0,2,3.5,5,65.0,8450,...,856,856,854,0,1710,548,0,0,0,2566
1,3,1,6,1,1976.0,2,2.5,31,80.0,9600,...,1262,1262,0,0,1262,460,298,0,0,2524
2,3,1,6,1,2001.0,2,3.5,6,68.0,11250,...,920,920,866,0,1786,608,0,0,0,2706
3,3,1,7,1,1998.0,3,2.0,36,60.0,9550,...,756,961,756,0,1717,642,0,0,0,2473
4,4,1,9,1,2000.0,3,3.5,8,84.0,14260,...,1145,1145,1053,0,2198,836,192,0,0,3343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,3,1,7,1,1999.0,2,2.5,7,62.0,7917,...,953,953,694,0,1647,460,0,0,0,2600
1456,3,1,7,2,1978.0,2,3.0,22,85.0,13175,...,1542,2073,0,0,2073,500,349,0,0,3615
1457,4,1,9,2,1941.0,1,2.0,4,66.0,9042,...,1152,1188,1152,0,2340,252,0,0,2500,3492
1458,2,1,5,0,1950.0,1,2.0,14,68.0,9717,...,1078,1078,0,0,1078,240,366,0,0,2156


In [75]:
num_vars = discrete_vars + continuous_vars
num_vars_df = train.loc[:,[i for i in num_vars]]
X = num_vars_df
y = train.SalePrice
X = X.apply(pd.to_numeric, errors='coerce')
y = y.apply(pd.to_numeric, errors='coerce')
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
rfe = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=5)
model = DecisionTreeRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -28939.359 (2013.401)


In [85]:
# get the dataset
def get_dataset():
    X = num_vars_df
    Y = train.SalePrice
    X = X.apply(pd.to_numeric, errors='coerce')
    Y = Y.apply(pd.to_numeric, errors='coerce')
    X.fillna(0, inplace=True)
    Y.fillna(0, inplace=True)
    return X, Y
# get a list of models to evaluate
def get_models():
    models = dict()
    for i in range(2,10):
        rfe = RFE(estimator=DecisionTreeClassifier(),n_features_to_select=i)
        model = DecisionTreeClassifier()
        pipeline = Pipeline(steps=[('s',rfe),('m',model)])
        models[str(i)] = pipeline
    return models
#evaluate a give model using cross-validation
def evaluate_model(model, X, Y):
    cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
    scores = cross_val_score(model,X,Y,scoring='accuracy',cv=cv,n_jobs=-1,error_score='raise')
    return scores

In [ ]:
X,Y = get_dataset()
models = get_models()
results, names = list(),list()
for name, model in models.items():
    scores = evaluate_model(model,X,Y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
pyplot.boxplot(results,labels=names,showmeans=True)
pyplot.show()

/Users/robertx/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/Users/robertx/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/Users/robertx/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


>2 0.007 (0.007)


/Users/robertx/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/Users/robertx/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/Users/robertx/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


>3 0.007 (0.007)


/Users/robertx/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/Users/robertx/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/Users/robertx/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
